In [37]:
from twython import Twython
from datetime import datetime
import re

In [3]:
APP_KEY = 'mPQKoRwd2Pb9qpQyQmyG5s8KR'
APP_SECRET = 'HLvIhusvfzDLKaRXY8CnZGP143kp3E3f2KqQBIEMfVL5mOxZjq'
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']


{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1612557999}}

In [34]:
data = twitter.search(
    q='#bitcoin',
    lang='en',
    result_type='popular',
    tweet_mode='extended',
    count="10"
)

{'statuses': [{'created_at': 'Fri Feb 05 07:15:55 +0000 2021',
   'id': 1357588751757627393,
   'id_str': '1357588751757627393',
   'full_text': 'Running #bitcoin https://t.co/W51ga3yrKb',
   'truncated': False,
   'display_text_range': [0, 16],
   'entities': {'hashtags': [{'text': 'bitcoin', 'indices': [8, 16]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [],
    'media': [{'id': 1357588748041506817,
      'id_str': '1357588748041506817',
      'indices': [17, 40],
      'media_url': 'http://pbs.twimg.com/media/EtcfxgYU0AEBD1r.jpg',
      'media_url_https': 'https://pbs.twimg.com/media/EtcfxgYU0AEBD1r.jpg',
      'url': 'https://t.co/W51ga3yrKb',
      'display_url': 'pic.twitter.com/W51ga3yrKb',
      'expanded_url': 'https://twitter.com/jack/status/1357588751757627393/photo/1',
      'type': 'photo',
      'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
       'small': {'w': 680, 'h': 531, 'resize': 'fit'},
       'medium': {'w': 816, 'h': 637, 'resize': 'fi

In [43]:
tweet = data['statuses'][1]
result = {
    '_id': tweet['id'],
    'text': tweet['full_text'],
    'user_id': tweet['user']['id'],
    'follower_count': tweet['user']['followers_count'],
    'verified': tweet['user']['verified'],
    'created_at': datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),
    'retweet_count': tweet['retweet_count'],
    'favorite_count': tweet['favorite_count'],
    'topics': [
        tag.lower()
        for tag in re.findall(r"#(\w*[0-9a-zA-Z]+\w*[0-9a-zA-Z])", tweet['full_text'])
    ]
}

result

{'text': 'German prosecutors have confiscated more than $60 million worth of #bitcoin from a fraudster. There’s only one problem: they can’t unlock the money because he won’t give them the password. @johnodonnell21 reports https://t.co/weuOUzqL62 1/4 https://t.co/Erd8fJhtID',
 'user_id': 1652541,
 'follower_count': 23060368,
 'verified': True,
 'created_at': datetime.datetime(2021, 2, 5, 13, 59, 8),
 'retweet_count': 2580,
 'favorite_count': 6320,
 'topics': ['bitcoin']}

In [46]:
import pymongo

client = pymongo.MongoClient(
    "mongodb://localhost:27017/",
    username='root',
    password='bigbrain'
)

In [49]:
scrapperdb = client['scrapper']
tweets = scrapperdb['tweets']

In [64]:
tweets.update({
    '_id': result['_id']
}, result, upsert=True)

<ipython-input-64-53a015f559d2>:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  tweets.update({


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [54]:
tweets.ensure_index([
    ('created_at', pymongo.DESCENDING),
    ('topics', pymongo.ASCENDING)
])

<ipython-input-54-39d8aaf8a241>:1: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  tweets.ensure_index([


'created_at_-1_topics_1'

In [59]:
list(tweets.find(
    {'topics': {'$in': ['bitcoin']}}
))

[{'_id': ObjectId('601db6a49c687ce76749443e'),
  'text': 'German prosecutors have confiscated more than $60 million worth of #bitcoin from a fraudster. There’s only one problem: they can’t unlock the money because he won’t give them the password. @johnodonnell21 reports https://t.co/weuOUzqL62 1/4 https://t.co/Erd8fJhtID',
  'user_id': 1652541,
  'follower_count': 23060368,
  'verified': True,
  'created_at': datetime.datetime(2021, 2, 5, 13, 59, 8),
  'retweet_count': 2580,
  'favorite_count': 6320,
  'topics': ['bitcoin']}]

In [60]:
tweet

{'created_at': 'Fri Feb 05 13:59:08 +0000 2021',
 'id': 1357690223187161088,
 'id_str': '1357690223187161088',
 'full_text': 'German prosecutors have confiscated more than $60 million worth of #bitcoin from a fraudster. There’s only one problem: they can’t unlock the money because he won’t give them the password. @johnodonnell21 reports https://t.co/weuOUzqL62 1/4 https://t.co/Erd8fJhtID',
 'truncated': False,
 'display_text_range': [0, 240],
 'entities': {'hashtags': [{'text': 'bitcoin', 'indices': [67, 75]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'johnodonnell21',
    'name': "John O'Donnell",
    'id': 175029793,
    'id_str': '175029793',
    'indices': [189, 204]}],
  'urls': [{'url': 'https://t.co/weuOUzqL62',
    'expanded_url': 'https://reut.rs/39QG265',
    'display_url': 'reut.rs/39QG265',
    'indices': [213, 236]}],
  'media': [{'id': 1357690127405973504,
    'id_str': '1357690127405973504',
    'indices': [241, 264],
    'media_url': 'http://pbs.twimg.com/me